In [ ]:
import pandas as pd
import numpy as np

In [ ]:
compustat = pd.read_csv("Compustat19612018.csv")
crsp = pd.read_csv("CRSP2007_2018.csv")
linking_table = pd.read_csv("CompustatMergedDatabase.csv")

In [ ]:
# We need link gvkey to compustat if compustat.datadate is between linking_table.LINKDT and linking_table.LINKENDDT
linking_table.head()

In [ ]:
# function to change E, which means the company still alive today, in linking table to today()
from datetime import date
def change_LINKENDDT(x):
    if(x=='E'): x = date.today()
    else : pass
    return x
change_LINKENDDT_vectorize = np.vectorize(change_LINKENDDT)

In [ ]:
# covert date using pandas to_date()
linking_table['LINKENDDT_date'] = pd.to_datetime(change_LINKENDDT_vectorize(linking_table['LINKENDDT']),infer_datetime_format=True)
linking_table['LINKDT_date'] = pd.to_datetime(linking_table['LINKDT'],infer_datetime_format=True)
compustat['datadate_date'] = pd.to_datetime(compustat['datadate'],infer_datetime_format=True)

In [ ]:
# link compustat and linking_table on gvkey of both
compustat_linking = compustat.merge(linking_table,how='outer',left_on=['gvkey'],right_on=['gvkey'],validate='many_to_many')

In [ ]:
# drop if compustat dates are between effective start LINKDT_date and end LINKENDDT_date
compustat_linking_drop = compustat_linking[(compustat_linking.datadate_date >= compustat_linking.LINKDT_date) & (compustat_linking.datadate_date <=  compustat_linking.LINKENDDT_date)]

In [ ]:
# check the shape of new dataframe
print(compustat_linking.shape)
print(compustat_linking_drop.shape)
print(compustat.shape)

In [ ]:
# merge compustat_linking_drop and crsp using to LPERMCO from linking table and PERMNO fron crsp to compustat_crsp
compustat_crsp = compustat_linking_drop.merge(crsp,how='outer',left_on=['LPERMCO'],right_on=['PERMNO'])

In [ ]:
# check shape of compustat_crsp
compustat_crsp.shape

In [ ]:
# check compustat_crsp
compustat_crsp.head(50)

### Ignore below

In [ ]:
#df = compustat_linked.drop(compustat_linked[(compustat_linked.datadate >= compustat_linked.LINKDT) & (compustat_linked.datadate <=  compustat_linked.LINKENDDT)].index)

In [ ]:
#compustat.groupby(['gvkey','datadate']).size().sort_values(ascending=False).head()

In [ ]:
print(linking_table['gvkey'].size)
print(linking_table['gvkey'].nunique())

print(compustat['gvkey'].size)
print(compustat['gvkey'].nunique())